# Evaluation Harness (CPU)

## Why Evaluate on CPU?

Before training, we want to:
1. **Baseline perplexity:** Measure base model performance
2. **Sample generation:** See what the base model produces
3. **Compare post-training:** Same metrics after finetuning

Running evaluation on CPU is slow but:
- No GPU needed for initial checks
- Helps validate the pipeline
- Can run locally before GPU training

## Perplexity Approximation

Perplexity measures how "surprised" the model is by the data:
- Lower = better (model predicts data well)
- Formula: exp(mean(negative_log_likelihood))

We'll compute it on a small validation slice for speed.

### What You'll See When Running This:

1. **Dataset Loading:** Downloads/loads your dataset from Hugging Face Hub
   - Train split: 456 samples
   - Validation split: 25 samples

2. **Model Download (First Time Only):**
   - Downloads Mistral-7B (~15GB total)
   - 3 model files (model-00001/02/03-of-00003.safetensors)
   - This only happens once - files are cached locally
   - **This is what you're seeing now!** ⬇️

3. **Model Loading:**
   - Loads the 7B parameter model into RAM
   - Uses CPU (no GPU needed, but slower)
   - Takes a few minutes

4. **Perplexity Calculation:**
   - Processes validation samples one by one
   - Computes how well model predicts each token
   - Averages across all samples
   - **This will take 10-30 minutes on CPU** (be patient!)

### Why So Slow on CPU?
- 7B parameters = billions of calculations per sample
- CPU has fewer cores than GPU
- This is why we only do this once for baseline, then use GPU for training


In [1]:
# Check if GPU cuda is available
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset

if torch.cuda.is_available():
    print("✅ GPU detected: ", torch.cuda.get_device_name(0))
else:
    print("⚠️  No GPU detected! This will be very slow or crash.")
    print("   Please enable GPU in Colab: Runtime → Change runtime type → GPU")


✅ GPU detected:  NVIDIA A100-SXM4-80GB


## Authentication for Private Dataset

**⚠️ IMPORTANT:** The dataset `Tuminha/frankenstein-fanfic-snippets` is **private**, so you must authenticate first!

### Option 1: Using .env file (Recommended for local development)

Create a `.env` file in your project root with:
```
HF_TOKEN=your_token_here
```

Or use any of these variable names:
- `HF_TOKEN`
- `HUGGINGFACE_HUB_TOKEN`
- `HUGGINGFACE_API_KEY`

**Note:** Make sure `.env` is in `.gitignore` (it already is)!

### Option 2: Interactive Login

Run the authentication cell below to login interactively:

```python
from huggingface_hub import login
login()  # Paste your token when prompted
```

### Option 3: Environment Variable (Colab/Cloud)

Set environment variable before running:
```python
import os
os.environ["HF_TOKEN"] = "your_token_here"
```

### Get Your Token
1. Go to: https://huggingface.co/settings/tokens
2. Click "New token"
3. Name it (e.g., "colab-access")
4. Select "Read" access
5. Copy the token

**Note:** The dataset uses Parquet format (not CSV) - this is fine! `load_dataset()` handles it automatically.


In [3]:
# Helper function to load dataset (Hub or local fallback)
def load_dataset_with_fallback(hub_id="Tuminha/frankenstein-fanfic-snippets", token=None):
    """
    Try to load dataset from Hub, with authentication if needed.
    Falls back to local CSV if Hub access fails.

    Args:
        hub_id: Hugging Face dataset ID
        token: Optional Hugging Face token (or use login() first)

    Returns:
        DatasetDict with 'train' and 'validation' splits
    """
    from datasets import Dataset, DatasetDict
    from sklearn.model_selection import train_test_split
    import pandas as pd
    import os

    # Try Hub first (with authentication if needed)
    try:
        # Get token from environment or cached login
        from huggingface_hub import HfFolder
        import os

        # Try to get token from various sources (in order of priority)
        hf_token = token  # 1. Use provided token first

        # 2. Try to load from .env file
        if not hf_token:
            try:
                from dotenv import load_dotenv
                load_dotenv()  # Load .env file if it exists
                hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN") or os.getenv("HUGGINGFACE_API_KEY")
            except ImportError:
                # python-dotenv not installed, skip .env loading
                pass
            except Exception:
                # .env file doesn't exist or other error, continue
                pass

        # 3. Try environment variables (already loaded from system/env)
        if not hf_token:
            hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN") or os.getenv("HUGGINGFACE_API_KEY")

        # 4. Try to get from HfFolder (cached login via login())
        if not hf_token:
            try:
                hf_token = HfFolder.get_token()
            except Exception:
                pass

        # Load dataset with explicit token
        if hf_token:
            token_source = "provided parameter"
            if not token:  # If not provided as parameter, figure out source
                if os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN") or os.getenv("HUGGINGFACE_API_KEY"):
                    token_source = "environment variable (.env or system)"
                else:
                    token_source = "cached login (HfFolder)"
            print(f"✅ Using authentication token from: {token_source} (length: {len(hf_token)})")
            dset = load_dataset(hub_id, token=hf_token)
        else:
            print("⚠️  No token found. Trying without explicit token (may use cached)...")
            dset = load_dataset(hub_id)  # Will try to use cached token

        print("✅ Loaded dataset from Hugging Face Hub")
        print(f"   Train: {len(dset['train'])}, Validation: {len(dset['validation'])}")
        return dset

    except Exception as e:
        error_msg = str(e).lower()
        error_type = type(e).__name__

        print(f"⚠️  Could not load from Hub: {error_type}")

        # Check if it's an authentication/access issue
        if "not found" in error_msg or "cannot be accessed" in error_msg or "401" in error_msg:
            print("\n🔐 AUTHENTICATION REQUIRED")
            print("   The dataset is private. You need to authenticate first.")
            print("\n   Run this in a cell above:")
            print("   ```python")
            print("   from huggingface_hub import login")
            print("   login()  # Enter your token when prompted")
            print("   ```")
            print("\n   Get your token from: https://huggingface.co/settings/tokens")
            print("   (Create a token with 'read' access)")
            print("\n   After authenticating, run this cell again.")
            print("\n   Alternatively, you can pass a token directly:")
            print("   ```python")
            print("   dset = load_dataset_with_fallback(token='your_token_here')")
            print("   ```")

        print("\n   Falling back to local CSV...")

        # Fallback: Try multiple possible CSV paths
        possible_paths = [
            "../data/processed/frankenstein_cleaned.csv",  # Local relative
            "data/processed/frankenstein_cleaned.csv",     # Colab root
            "/content/data/processed/frankenstein_cleaned.csv",  # Colab content
            "./data/processed/frankenstein_cleaned.csv",   # Current dir
        ]

        df = None
        used_path = None

        for csv_path in possible_paths:
            try:
                if os.path.exists(csv_path):
                    df = pd.read_csv(csv_path)
                    used_path = csv_path
                    print(f"   ✅ Found CSV at: {csv_path}")
                    break
            except Exception:
                continue

        if df is None:
            print(f"\n❌ CSV file not found in any of these locations:")
            for p in possible_paths:
                print(f"   - {p}")
            print("\n   💡 SOLUTIONS:")
            print("   1. Authenticate to Hub (recommended):")
            print("      from huggingface_hub import login")
            print("      login()")
            print("\n   2. Upload CSV to Colab:")
            print("      from google.colab import files")
            print("      files.upload()  # Upload frankenstein_cleaned.csv")
            print("\n   3. Download dataset files manually from Hub and load them")

            raise FileNotFoundError(
                "Could not access dataset. Please authenticate to Hub or provide CSV file.\n"
                "The dataset exists but is private - authentication is required."
            )

        print(f"   Loaded CSV: {len(df)} rows from {used_path}")

        # Create train/val split (same as notebook 02: 5% validation, seed=42)
        train_df, val_df = train_test_split(df, test_size=0.05, random_state=42)

        dset = DatasetDict({
            'train': Dataset.from_pandas(train_df),
            'validation': Dataset.from_pandas(val_df)
        })
        print(f"✅ Created dataset from local CSV: {len(dset['train'])} train, {len(dset['validation'])} val")
        return dset

# Load dataset
# NOTE: If you get authentication error, run this first:
# from huggingface_hub import login
# login()  # Enter your token
# Then run this cell again
dset = load_dataset_with_fallback()


✅ Using authentication token from: cached login (HfFolder) (length: 37)


README.md:   0%|          | 0.00/383 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/278k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/20.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/456 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25 [00:00<?, ? examples/s]

✅ Loaded dataset from Hugging Face Hub
   Train: 456, Validation: 25


In [4]:
# === TODO (you code this) ===
# Compute perplexity for a small validation slice using a CPU-friendly model.
# NOTE: Mistral-7B is too large for CPU! Use DistilGPT-2 instead.
# Hints:
#   - Load model and tokenizer (use "distilgpt2" for CPU)
#   - Tokenize the dataset first (it's not tokenized yet!)
#   - For each sample, compute negative log-likelihood of tokens
#   - Sum NLL, divide by total tokens, then exp() for perplexity
# Acceptance:
#   - prints ppl_base on ~25 validation samples (all of them)

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset
from dotenv import load_dotenv
import os
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_API_KEY") or os.getenv("HF_TOKEN")

def cpu_perplexity_estimate(base_model: str, dataset, n_samples: int=25):
    """
    Estimate perplexity on CPU using a smaller model (DistilGPT-2).

    NOTE: Mistral-7B requires ~28GB RAM and will crash on most CPUs.
    We use DistilGPT-2 as a proxy baseline here.
    Real Mistral baseline will be computed on GPU in notebook 11.

    Args:
        base_model: Model name (use "distilgpt2" for CPU)
        dataset: Validation dataset (not tokenized yet)
        n_samples: Number of samples to evaluate
    """
    print(f"Loading model {base_model} on CPU...")
    print("⚠️  Using DistilGPT-2 as proxy (Mistral-7B too large for CPU)")

    model = AutoModelForCausalLM.from_pretrained(base_model)
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.pad_token = tokenizer.eos_token  # Set padding token
    model.eval()

    # Limit samples
    n_samples = min(n_samples, len(dataset))
    print(f"Computing perplexity on {n_samples} samples...")
    print("⚠️  This may take 5-15 minutes. Be patient!")

    import time
    start_time = time.time()
    total_nll = 0.0
    total_tokens = 0

    with torch.no_grad():
        for i, sample in enumerate(dataset.select(range(n_samples))):
            # Tokenize the text
            text = sample['text']
            encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            input_ids = encoded['input_ids']

            # Forward pass to get logits
            outputs = model(input_ids, labels=input_ids)

            # Loss is already negative log-likelihood per token (averaged)
            nll = outputs.loss.item() * input_ids.size(1)

            total_nll += nll
            total_tokens += input_ids.size(1)

            if (i + 1) % 5 == 0:
                print(f"  Processed {i + 1}/{n_samples} samples...")

    # Compute perplexity
    avg_nll = total_nll / total_tokens
    perplexity = torch.exp(torch.tensor(avg_nll)).item()

    print(f"\n✅ Baseline Perplexity (DistilGPT-2): {perplexity:.2f}")
    print(f"   (computed on {n_samples} samples, {total_tokens} tokens)")
    print(f"\n📝 Note: This is DistilGPT-2 baseline, not Mistral.")
    print(f"   Real Mistral baseline will be computed on GPU in notebook 11.")

    return perplexity

# Load dataset and compute baseline
dset = load_dataset("Tuminha/frankenstein-fanfic-snippets")
# Use DistilGPT-2 instead of Mistral (Mistral too large for CPU)
cpu_perplexity_estimate("mistralai/Mistral-7B-Instruct-v0.2", dset['validation'], n_samples=25)


Loading model mistralai/Mistral-7B-Instruct-v0.2 on CPU...
⚠️  Using DistilGPT-2 as proxy (Mistral-7B too large for CPU)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Computing perplexity on 25 samples...
⚠️  This may take 5-15 minutes. Be patient!
  Processed 5/25 samples...
  Processed 10/25 samples...
  Processed 15/25 samples...
  Processed 20/25 samples...
  Processed 25/25 samples...

✅ Baseline Perplexity (DistilGPT-2): 10.55
   (computed on 25 samples, 5828 tokens)

📝 Note: This is DistilGPT-2 baseline, not Mistral.
   Real Mistral baseline will be computed on GPU in notebook 11.


10.547439575195312

## 💾 Saving Changes from Colab to GitHub

### Method 1: Download and Commit Locally
1. In Colab: **File → Download → Download .ipynb**
2. Save the downloaded file
3. On your local machine, replace the notebook file
4. Commit and push:
   ```bash
   git add notebooks/04_eval_harness_cpu.ipynb
   git commit -m "Update notebook 04: Add perplexity results"
   git push
   ```

### Method 2: Use Git in Colab (Advanced)
```python
# Install git (if needed)
!apt-get install git -y

# Configure git (one-time setup)
!git config --global user.name "Your Name"
!git config --global user.email "your.email@example.com"

# Clone repo (if not already)
!git clone https://github.com/Tuminha/Frankenstein-LLM-Fine-Tuning-with-Mistral.git

# After making changes:
!cd Frankenstein-LLM-Fine-Tuning-with-Mistral && git add notebooks/04_eval_harness_cpu.ipynb
!cd Frankenstein-LLM-Fine-Tuning-with-Mistral && git commit -m "Update notebook 04"
!cd Frankenstein-LLM-Fine-Tuning-with-Mistral && git push
```

**Note:** For Method 2, you'll need to set up a GitHub Personal Access Token for authentication.


## Mistral-7B Perplexity (GPU/Colab)

**⚠️ Run this cell in Google Colab with GPU enabled!**

This version uses Mistral-7B for the real baseline. It requires:
- GPU runtime in Colab (Runtime → Change runtime type → GPU)
- ~15GB GPU memory (T4 works fine)
- Much faster than CPU (minutes instead of hours)

### ⏱️ Expected Runtime
- **Model download:** 5-10 minutes (first time only, ~15GB)
- **Model loading:** 1-2 minutes
- **Perplexity on 25 samples:** 5-15 minutes
- **Total:** ~10-30 minutes

### ⚠️ If It Seems "Stuck"
The perplexity calculation processes samples one by one. Each sample takes ~20-40 seconds. If you don't see output for a few minutes, it's likely still running. Check:
- GPU memory usage (should be high)
- Colab runtime indicator (should show activity)
- Progress prints appear every 5 samples

**It's normal for this to take 10-15 minutes!** Be patient.


In [6]:
# === Mistral-7B Perplexity for GPU/Colab ===
# Copy this cell to Google Colab and run with GPU enabled!
# Runtime → Change runtime type → GPU (T4)

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset

def gpu_perplexity_estimate_mistral(base_model: str, dataset, n_samples: int=25):
    """
    Estimate perplexity using Mistral-7B on GPU (for Colab).

    Args:
        base_model: Model name ("mistralai/Mistral-7B-Instruct-v0.2")
        dataset: Validation dataset (not tokenized)
        n_samples: Number of samples to evaluate
    """
    # Check for GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("⚠️  WARNING: No GPU detected! This will be very slow or crash.")
        print("   Please enable GPU in Colab: Runtime → Change runtime type → GPU")
        return None

    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"Loading model {base_model} on {device}...")

    # Load model on GPU with bfloat16 (saves memory)
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        dtype=torch.bfloat16,  # Use bfloat16 for efficiency (fixed: dtype instead of torch_dtype)
        device_map="auto"  # Automatically place on GPU
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.pad_token = tokenizer.eos_token
    model.eval()

    # Limit samples
    n_samples = min(n_samples, len(dataset))
    print(f"Computing perplexity on {n_samples} samples...")
    print("⚠️  This may take 5-15 minutes. Each sample takes ~20-40 seconds.")
    print("   Progress will be printed every 2 samples. Be patient!\n")

    import time
    start_time = time.time()
    total_nll = 0.0
    total_tokens = 0

    with torch.no_grad():
        for i, sample in enumerate(dataset.select(range(n_samples))):
            # Tokenize the text
            text = sample['text']
            encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            input_ids = encoded['input_ids'].to(device)  # Move to GPU

            # Forward pass to get logits
            outputs = model(input_ids, labels=input_ids)

            # Loss is already negative log-likelihood per token (averaged)
            nll = outputs.loss.item() * input_ids.size(1)

            total_nll += nll
            total_tokens += input_ids.size(1)

            # Print progress every 2 samples with time estimate
            if (i + 1) % 2 == 0:
                elapsed = time.time() - start_time
                if i > 0:
                    avg_time = elapsed / (i + 1)
                    remaining = avg_time * (n_samples - i - 1)
                    print(f"  Processed {i + 1}/{n_samples} samples... (~{remaining:.0f}s remaining)")
                else:
                    print(f"  Processed {i + 1}/{n_samples} samples...")

    # Compute perplexity
    avg_nll = total_nll / total_tokens
    perplexity = torch.exp(torch.tensor(avg_nll)).item()

    elapsed_time = time.time() - start_time
    print(f"\n✅ Baseline Perplexity (Mistral-7B): {perplexity:.2f}")
    print(f"   (computed on {n_samples} samples, {total_tokens} tokens)")
    print(f"   Device: {device}")
    print(f"   Time taken: {elapsed_time/60:.1f} minutes ({elapsed_time:.0f} seconds)")

    return perplexity

# Load dataset and compute baseline
# Uncomment and run in Colab:
dset = load_dataset("Tuminha/frankenstein-fanfic-snippets")
gpu_perplexity_estimate_mistral("mistralai/Mistral-7B-Instruct-v0.2", dset['validation'], n_samples=25)


✅ GPU detected: NVIDIA A100-SXM4-80GB
   GPU Memory: 85.2 GB
Loading model mistralai/Mistral-7B-Instruct-v0.2 on cuda...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Computing perplexity on 25 samples...
⚠️  This may take 5-15 minutes. Each sample takes ~20-40 seconds.
   Progress will be printed every 2 samples. Be patient!

  Processed 2/25 samples... (~9s remaining)
  Processed 4/25 samples... (~4s remaining)
  Processed 6/25 samples... (~3s remaining)
  Processed 8/25 samples... (~2s remaining)
  Processed 10/25 samples... (~2s remaining)
  Processed 12/25 samples... (~1s remaining)
  Processed 14/25 samples... (~1s remaining)
  Processed 16/25 samples... (~1s remaining)
  Processed 18/25 samples... (~1s remaining)
  Processed 20/25 samples... (~0s remaining)
  Processed 22/25 samples... (~0s remaining)
  Processed 24/25 samples... (~0s remaining)

✅ Baseline Perplexity (Mistral-7B): 10.55
   (computed on 25 samples, 5828 tokens)
   Device: cuda
   Time taken: 0.0 minutes (2 seconds)


10.547740936279297

## Mistral-7B Generation (GPU/Colab)

**⚠️ Run this cell in Google Colab with GPU enabled!**

Generate samples with Mistral-7B on GPU. Much faster and better quality than DistilGPT-2.


In [7]:
# === Mistral-7B Generation for GPU/Colab ===
# Copy this cell to Google Colab and run with GPU enabled!

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def gpu_sample_generate_mistral(base_model: str, prompts: list, max_new_tokens: int=100):
    """
    Generate text samples using Mistral-7B on GPU (for Colab).

    Args:
        base_model: Model name ("mistralai/Mistral-7B-Instruct-v0.2")
        prompts: List of prompt strings
        max_new_tokens: Maximum tokens to generate
    """
    # Check for GPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        print("⚠️  WARNING: No GPU detected! This will be very slow or crash.")
        print("   Please enable GPU in Colab: Runtime → Change runtime type → GPU")
        return None

    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"Loading model {base_model} on {device}...\n")

    # Load model on GPU with bfloat16
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.pad_token = tokenizer.eos_token
    model.eval()

    for i, prompt in enumerate(prompts, 1):
        print(f"Prompt {i}: {prompt}")
        print("Generating (GPU - this should be fast)...")

        # Tokenize prompt with attention_mask
        encoded = tokenizer(
            prompt,
            return_tensors="pt",
            padding=False,
            truncation=True,
            max_length=512
        ).to(device)

        # Generate with attention_mask
        with torch.no_grad():
            outputs = model.generate(
                encoded['input_ids'],
                attention_mask=encoded['attention_mask'],  # Pass attention_mask
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.2,  # Reduce repetition
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        continuation = generated_text[len(prompt):].strip()

        print(f"Continuation: {continuation}\n")
        print("-" * 80 + "\n")

# Test generation
# Uncomment and run in Colab:
prompts = [
    "It was on a dreary night of November that",
    "The monster gazed upon his creator with"
]
gpu_sample_generate_mistral("mistralai/Mistral-7B-Instruct-v0.2", prompts, max_new_tokens=100)


✅ GPU detected: NVIDIA A100-SXM4-80GB
Loading model mistralai/Mistral-7B-Instruct-v0.2 on cuda...



`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Prompt 1: It was on a dreary night of November that
Generating (GPU - this should be fast)...
Continuation: I beheld the apparition. The rain fell in torrents, except when it was driven back by furious gusts of wind which swept over the deserted streets and through the leafless trees like ghostly processions. As the storm raged around me with an intensity such as I had never known before, I sat alone in my study, brooding over the dark secret which had been revealed to me but a few days previously.

The fire crackled m

--------------------------------------------------------------------------------

Prompt 2: The monster gazed upon his creator with
Generating (GPU - this should be fast)...
Continuation: a mixture of fear and fascination. “What is this thing you have made?” it asked, its voice rumbling like distant thunder.

Dr. Frankenstein stepped back, his heart pounding in his chest. He had never seen the creature up close before – only from afar as it lurched through the forest, s

## Sample Generation (CPU)

Generating text on CPU is very slow, but useful for:
- Seeing base model outputs
- Validating the generation pipeline
- Comparing before/after training

We'll generate short continuations (60 tokens max) for a few prompts.


In [8]:
# === TODO (you code this) ===
# Generation wrapper using DistilGPT-2 on CPU for 1-2 short prompts.
# NOTE: Using DistilGPT-2 instead of Mistral (Mistral too large for CPU).
# Hints:
#   - Load model and tokenizer (use "distilgpt2")
#   - Use model.generate() with max_new_tokens
#   - Decode and print outputs
#   - Warn about CPU latency
# Acceptance:
#   - prints 2 short continuations; warns about CPU latency

def cpu_sample_generate(base_model: str, prompts: list, max_new_tokens: int=60):
    """
    Generate text samples on CPU using DistilGPT-2 (Mistral too large for CPU).

    Args:
        base_model: Model name (use "distilgpt2" for CPU)
        prompts: List of prompt strings
        max_new_tokens: Maximum tokens to generate
    """
    print(f"⚠️  Using {base_model} for CPU generation (Mistral-7B too large)")
    print("   Real Mistral generation will be done on GPU in notebook 11.")
    print("   Note: DistilGPT-2 wasn't trained on Frankenstein text, so outputs")
    print("   won't match the style. This is just for testing the pipeline.\n")

    model = AutoModelForCausalLM.from_pretrained(base_model)
    tokenizer = AutoTokenizer.from_pretrained(base_model)

    # Fix: Set pad_token properly (DistilGPT-2 doesn't have one by default)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model.eval()

    for i, prompt in enumerate(prompts, 1):
        print(f"Prompt {i}: {prompt}")
        print("Generating (CPU - this may take 30-60 seconds)...")

        # Fix: Tokenize with attention_mask (like notebook 3)
        # This explicitly creates attention_mask to avoid the warning
        encoded = tokenizer(
            prompt,
            return_tensors="pt",
            padding=False,  # No padding needed for single prompt
            truncation=True,
            max_length=512
        )

        # Generate with attention_mask (fixes the warning)
        with torch.no_grad():
            outputs = model.generate(
                encoded['input_ids'],
                attention_mask=encoded['attention_mask'],  # Fix: Pass attention_mask
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.8,  # Slightly higher for more variety
                top_p=0.9,  # Nucleus sampling
                repetition_penalty=1.2,  # Reduce repetition
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        continuation = generated_text[len(prompt):].strip()

        print(f"Continuation: {continuation}\n")
        print("-" * 80 + "\n")

# Test generation
prompts = [
    "It was on a dreary night of November that",
    "The monster gazed upon his creator with"
]
cpu_sample_generate("distilgpt2", prompts, max_new_tokens=60)


⚠️  Using distilgpt2 for CPU generation (Mistral-7B too large)
   Real Mistral generation will be done on GPU in notebook 11.
   Note: DistilGPT-2 wasn't trained on Frankenstein text, so outputs
   won't match the style. This is just for testing the pipeline.



config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Prompt 1: It was on a dreary night of November that
Generating (CPU - this may take 30-60 seconds)...
Continuation: I had been working with for about two years.
"I came to the conclusion, this is my first time doing so and it's all part 'em! We're still waiting here because our last three months have just gone by without any explanation."

--------------------------------------------------------------------------------

Prompt 2: The monster gazed upon his creator with
Generating (CPU - this may take 30-60 seconds)...
Continuation: a giddy grin and then said: "What the fuck am I going to do about this, Mr. Gorman?"
"Mr! Do you want me to get away with it all? You're not an idiot!" The creature laughed at him for what he did but quickly turned on himself

--------------------------------------------------------------------------------

